In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [14]:
df = pd.read_csv('demand_data.csv')

In [15]:
res = df['choice'].apply(lambda c: ['x1_prod' + str(c), 'x2_prod' + str(c), 'x3_prod' + str(c)])
df[['prod_char1','prod_char2','prod_char3']] = [df.loc[i, res[i]].values for i in range(df.shape[0])]

In [41]:
denom = np.zeros(5000)
num = np.zeros((5000, 3))
for i in range(4):
    cols = ['x1_prod' + str(i), 'x2_prod' + str(i), 'x3_prod' + str(i)]


    df['rhs_frac'+str(i)] = np.exp(np.matmul(df[cols], beta))
    denom += df['rhs_frac'+str(i)].values
    num += df[cols].apply(lambda x: np.asarray(x) * np.asarray(df['rhs_frac'+str(i)])).values

In [50]:
def calculateScore(beta):
    denom = np.zeros(5000)
    num = np.zeros((5000, 3))
    for i in range(4):
        cols = ['x1_prod' + str(i), 'x2_prod' + str(i), 'x3_prod' + str(i)]
        
        
        df['rhs_frac'+str(i)] = np.exp(np.matmul(df[cols], beta))
        denom += df['rhs_frac'+str(i)].values
        num += df[cols].apply(lambda x: np.asarray(x) * np.asarray(df['rhs_frac'+str(i)])).values

    series_dif = (df[['prod_char1','prod_char2','prod_char3']].values - pd.DataFrame(num).divide(denom, axis = 0).values)
    return series_dif.sum(axis = 0)

In [56]:
def MLE_estimation(beta):
    rhs = np.zeros(5000)
    for i in range(4):
        cols = ['x1_prod' + str(i), 'x2_prod' + str(i), 'x3_prod' + str(i)]
        df['rhs_frac'+str(i)] = np.exp(np.matmul(df[cols], beta))
        rhs += df['rhs_frac'+str(i)].values

    return -np.sum(np.matmul(df[['prod_char1','prod_char2','prod_char3']], beta) - np.log(rhs))

In [58]:
MLE_calc = minimize(MLE_estimation, [1., 1., 1.], method = 'SLSQP', tol=1e-9)
beta_mle = MLE_calc.x

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [59]:
beta_mle

array([-2.4925639 ,  3.06875208,  1.99283749])

In [60]:
calculateScore(beta_mle)

array([ 3.24844658e-04, -1.51474090e-05,  3.96477657e-04])

In [79]:
df['choice_market'] = df['choice'].apply(lambda x: str(x)) + '_' + df['marketindex'].apply(lambda x: str(x))

In [81]:
choice_dict = dict(df['choice_market'].value_counts())
df['outcome'] = np.log(df['choice_market'].apply(lambda c: choice_dict[c]/choice_dict[str(0)+"_"+c.split("_")[1]] if str(0)+"_"+c.split("_")[1] in choice_dict.keys() else np.nan))

In [86]:
df_reg = df[df['outcome'].apply(lambda x: not pd.isnull(x))]

In [87]:
lr = LinearRegression()
lr.fit(df_reg[['prod_char1','prod_char2','prod_char3']], df_reg['outcome'])
lr.coef_

array([-2.26806387,  2.87693902,  1.83384677])

In [88]:
calculateScore(lr.coef_)

array([-870.45967358,  -38.17642343, -934.94875622])